#  Used Car Dealer

**Author:**
[Anthony Strittmatter](http://www.anthonystrittmatter.com)

You take the role of a used car dealer. You have a database with prices and characteristics of used cars you sold previously (*use\_car\_database.csv*). You want to use this data to predict the prices of the used cars that are currently in your garage. For these cars you have a data set containing the characteristics (*new\_used\_cars.csv*), but not the prices. The characteristics that you observe in the data sets are described in the file *variable\_description.xlsx*.

Your task is to predict the used car prices of the used cars that are currently in your garage using any method of your choice.

## Load Packages and Data

In [22]:
########################  Load Packages and Data  ########################

# Load packages
library(rpart)
library(rpart.plot)
library(grf)
library(glmnet)
library(nnet)

# Load data
cars <- read.csv("used_car_database.csv", sep = ",")
new_cars <- read.csv("new_used_cars.csv", sep = ",")

print('Packages and data successfully loaded.')

#############################################################################

[1] "Packages and data successfully loaded."


## Inspect Data

In [23]:
########################  Describe Old Data  ########################

# Print first few rows of old data
head(cars)

# Number of observations
print(paste0('Old data: ',nrow(cars),' observations'))

######################################################################

id,sales_price,mercedes_c,vw_golf,vw_passat,bmw_320,opel_astra,diesel,other_car_owner,pm_green,...,mile_40,mile_50,mile_100,mile_150,mileage,mileage2,age_3,age_6,age_car_years,age_car_years2
52927,12.060,0,0,1,0,0,1,2,1,...,1,1,1,0,149.000,22201.000,1,1,9.0,81.00
49185,21.121,0,0,1,0,0,1,1,1,...,1,0,0,0,42.000,1764.000,0,0,1.2,1.44
64639,31.700,0,0,0,1,0,0,1,1,...,0,0,0,0,14.300,204.490,0,0,1.7,2.89
11372,14.240,1,0,0,0,0,0,2,1,...,1,1,0,0,56.161,3154.058,1,1,6.4,40.96
7593,20.350,0,0,1,0,0,1,1,1,...,1,1,1,0,101.482,10298.596,1,0,4.3,18.49
89992,11.679,0,1,0,0,0,0,2,1,...,1,1,0,0,71.544,5118.544,1,1,7.0,49.00


[1] "Old data: 48976 observations"


In [24]:
########################  Describe Old Data  ########################

# Print first few rows of new data
head(new_cars)

# Number of observations
print(paste0('New data: ',nrow(new_cars),' observations'))

######################################################################

id,mercedes_c,vw_golf,vw_passat,bmw_320,opel_astra,diesel,other_car_owner,pm_green,private_seller,...,mile_40,mile_50,mile_100,mile_150,mileage,mileage2,age_3,age_6,age_car_years,age_car_years2
104720,0,1,0,0,0,0,0,0,1,...,1,0,0,0,40.000,1600.0000,0,0,2.5,6.25
32761,0,0,1,0,0,1,0,0,1,...,1,1,1,0,139.800,19544.0400,1,0,5.7,32.49
32601,0,0,1,0,0,0,0,1,0,...,1,1,1,0,134.000,17956.0000,1,1,8.9,79.21
53732,0,0,1,0,0,1,2,1,0,...,0,0,0,0,34.850,1214.5225,0,0,1.6,2.56
3655,0,1,0,0,0,1,1,1,0,...,1,1,0,0,90.142,8125.5802,1,0,4.9,24.01
98140,1,0,0,0,0,1,1,1,0,...,0,0,0,0,31.299,979.6274,0,0,1.2,1.44


[1] "New data: 1024 observations"


## Prepare Data

In [25]:
########################  Data Preparation  ########################

# Generate outcome and control variables
y <- as.matrix(cars[,2])
x <- as.matrix(cars[,-c(1:2)])
new_x <- as.matrix(new_cars[,-1])

print('Data is prepared.')

#############################################################################

[1] "Data is prepared."


**$\Rightarrow$ It is possible to add more non-linear and interaction terms.**

## Generate Training and Test Sample

In [26]:
########################  Training and Test Samples  ########################

set.seed(1001)

# Generate variable with the rows in training data
size <- floor(0.5 * nrow(cars))
training_set <- sample(seq_len(nrow(cars)), size = size)

print('Training and test samples created.')

#############################################################################

[1] "Training and test samples created."


## Predict Orange Juice Prices in Training Sample and Assess Model in Test Sample

### Lasso, Ridge, Elastic Net

In [27]:
########################  LASSO, Ridge, Elastic Net  ##############################

set.seed(27112019)
penalized.cv <- cv.glmnet(x[training_set,],y[training_set], 
                      type.measure = "mse", family = "gaussian", nfolds = 10, alpha = 0.5)


# Fitted values
pred_penalized <- predict(penalized.cv, newx = x, s = penalized.cv$lambda.min, alpha = 0.5)

# Calculate the MSE
MSE_penalized <- mean((y[-training_set] - pred_penalized[-training_set])^2)
R2_penalized <- round(1- MSE_penalized/var(y[-training_set]), digits = 3)

print(paste0("R-squared Penalized Regression: ", R2_penalized))
                                   
################################################################

[1] "R-squared Penalized Regression: 0.781"


### Tree

In [28]:
######################  Regression Tree  #######################

set.seed(27112019)
# Prepare data for tree estimator
outcome <- y[training_set]
tree_data <- data.frame(outcome, x[training_set,])

deep_tree <- rpart(formula = outcome ~ ., data = tree_data, method = "anova",
                xval = 10, y = TRUE, control = rpart.control(cp = 0.00002, minbucket=10))

# Optimal tree size
op.index <- which.min(deep_tree$cptable[, "xerror"])

## Select the Tree that Minimises CV-MSE
cp.vals <- deep_tree$cptable[op.index, "CP"]

# Prune the deep tree
pruned_tree <- prune(deep_tree, cp = cp.vals)

## Plot tree structure
#rpart.plot(pruned_tree,digits=3)

# Fitted values
predtree <- predict(pruned_tree, newdata= as.data.frame(x))

# Calculate the MSE
MSEtree <- mean((y[-training_set] - predtree[-training_set])^2)
R2tree <- round(1- MSEtree/var(y[-training_set]), digits = 3)

print(paste0("R-squared Tree: ", R2tree))

################################################################

[1] "R-squared Tree: 0.849"


### Random Forest

In [29]:
########################  Random Forest  #######################

set.seed(27112019)

rep <- 1000 # number of trees
cov <- 2/3 # share of covariates
frac <- 1/2 # fraction of subsample
min_obs <- 10 # max. size of terminal leaves in trees

# Build Forest
forest <- regression_forest(x[training_set,],y[training_set,], mtry = floor(cov*ncol(x)), sample.fraction = frac,
            num.trees = rep,min.node.size = min_obs, honesty=FALSE)

# Fitted values
predforest <- predict(forest, newdata=x)$predictions

# Calculate MSE
MSEforest <- mean((y[-training_set] - predforest[-training_set])^2)
R2forest <- round(1- MSEforest/var(y[-training_set]), digits = 3)

print(paste0("R-squared Forest: ", R2forest))

################################################################

[1] "R-squared Forest: 0.876"


## Select Favorite Model and Extrapolate to New Data

In [30]:
########################  Out-of-Sample Prediction  #######################

# Fitted values
# We select Random Forest because highest R-squared
new_prediction <- predict(forest, newdata=new_x)$predictions

print('Out-of-sample sales are predicted.')

###########################################################################

[1] "Out-of-sample sales are predicted."


## Store Out-of-Sample Predictions

In [32]:
########################  Store Results  #######################

id_new <- as.matrix(new_cars$id)

# Replace ??? with your group name
write.csv(cbind(id_new,new_prediction),"strittmatter.csv")

print('File is stored.')
print('Send your results to anthony.strittmatter@ensae.fr')

################################################################

[1] "File is stored."
[1] "Send your results to anthony.strittmatter@ensae.fr"
